In [6]:
import os, sys
from pathlib import Path
import json

WORK_DIR = Path.cwd().parent

sys.path.append(str(WORK_DIR))

from src import graph_gen
from src import ollama_manager


In [7]:
CVRR_DIR = Path("/multiverse/datasets/shared/cvrr_challange25/val")
VIDEO_DIR = WORK_DIR / "data/datasets/action-genome/Charades_v1_480"
video_ids = [{'video_id': "0A8CF"}]
video_path = VIDEO_DIR / "0A8CF.mp4"


In [8]:
print(video_path)


/home/lusha/star_code/data/datasets/action-genome/Charades_v1_480/0A8CF.mp4


In [9]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

frame_dir, frame_paths = graph_gen.extract_frames(video_path, 10, output_dir=".")
images = frame_paths
fig, axes = plt.subplots(1,  len(images), figsize=(20, 12))

for ax, img_path in zip(axes, images):
    img = mpimg.imread(img_path)
    ax.imshow(img)
    ax.axis('off')

plt.tight_layout()
plt.show()


KeyboardInterrupt: 

In [7]:
video_encodings = list(frames for frames in graph_gen.generate_frames(VIDEO_DIR, video_ids))
len(video_encodings)


Extracted frame at 0.10s: /tmp/tmpd46lbf97/frame_0000.jpg
Extracted frame at 3.58s: /tmp/tmpd46lbf97/frame_0001.jpg
Extracted frame at 7.16s: /tmp/tmpd46lbf97/frame_0002.jpg
Extracted frame at 10.74s: /tmp/tmpd46lbf97/frame_0003.jpg
Extracted frame at 14.32s: /tmp/tmpd46lbf97/frame_0004.jpg
Extracted frame at 17.91s: /tmp/tmpd46lbf97/frame_0005.jpg
Extracted frame at 21.49s: /tmp/tmpd46lbf97/frame_0006.jpg
Extracted frame at 25.07s: /tmp/tmpd46lbf97/frame_0007.jpg
Extracted frame at 28.65s: /tmp/tmpd46lbf97/frame_0008.jpg
Extracted frame at 32.13s: /tmp/tmpd46lbf97/frame_0009.jpg


1

In [8]:
video_encodings[0]


{'video_id': '0A8CF',
 'frames': [{'frame_id': 0,
   'encoding': '...'},
  {'frame_id': 1,
   'encoding': '...'},
  {'frame_id': 2,
   'encoding': '...'},
  {'frame_id': 3,
   'encoding': '...'},
  {'frame_id': 4,
   'encoding': '...'},
  {'frame_id': 5,
   'encoding': '...'},
  {'frame_id': 6,
   'encoding': '...'},
  {'frame_id': 7,
   'encoding': '...'},
  {'frame_id': 8,
   'encoding': '...'},
  {'frame_id': 9,
   'encoding': '...'}]}

In [4]:
sys_file_path = WORK_DIR / "data/prompts/graph_gen/system_prompt.txt"

sys_prompt = graph_gen._load_prompt_fromfile(sys_file_path)
# model_options = _load_model_options()
ollama_params = {
    "model": "gemma3:4b-it-qat",
    "system": sys_prompt,
    "stream": True,
    "options": {
        "num_ctx": 10240,
        "temperature": 0.1,
        "num_predict": 1024,
        "seed": 42,
    },
}

url = os.environ.get("OLLAMA_URL", "http://lusha_ollama:11435")
client = ollama_manager.OllamaRequestManager(url, ollama_params)


In [5]:
# pre-load the model
client.generate_completion("hello")


Hello there! I'm ready to put my scene graph analysis skills to work. Please provide me with an image, and I’ll do my best to break it down into a structured scene graph. Let’s see what we can discover! 

To help me give you the most useful analysis, could you also tell me:

*   **What kind of image are you sending?** (e.g., a photograph, a painting, a screenshot)
*   **Is there anything specific you're particularly interested in analyzing?** (e.g., the composition, the activities taking place, the objects involved)

Response at: 52.2 tk/s


"Hello there! I'm ready to put my scene graph analysis skills to work. Please provide me with an image, and I’ll do my best to break it down into a structured scene graph. Let’s see what we can discover! \n\nTo help me give you the most useful analysis, could you also tell me:\n\n*   **What kind of image are you sending?** (e.g., a photograph, a painting, a screenshot)\n*   **Is there anything specific you're particularly interested in analyzing?** (e.g., the composition, the activities taking place, the objects involved)"

In [15]:
prompt1 = """\
What's the image describing??

Think step by step.
"""

img = video_encodings[0]['frames'][0]['encoding']

msg1 = {
    "role": "user",
    "content": prompt1,
    "images": [img]
}
resp = client.ollama_completion_request(
            endpoint='chat',
            payload={
                "model": "gemma3:12b",
                "messages": [msg1],
                "options": {
                    "num_ctx": 10_000
                }
            }
        )


Okay, let's break down what's happening in this image step-by-step:

1. **Subject:** The main focus is a person, likely a young boy.
2. **Action:** The boy is actively writing or drawing on a paper that's in front of him. He's holding a writing utensil (pen/pencil) and appears focused.
3. **Position:** He's sitting directly on the floor, rather than on a chair.
4. **Setting:** The background is a simple, plain wall. There are some items on a table beside him, including what looks like a bag.

**Overall Description:**

The image shows a young boy sitting on the floor and writing or drawing on a piece of paper, likely studying or doing homework. The setting seems fairly simple and possibly quite humble.

Response at: 24.6 tk/s


In [18]:
images = [frame['encoding'] for frame in video_encodings[0]['frames']]

msg1 = {
    "role": "user",
    "content": prompt1,
    "images": images
}


resp = client.ollama_completion_request(
            endpoint='chat',
            payload={
                "model": "gemma3:12b-it-qat",
                "messages": [msg1],
                "options": {
                    "num_ctx": 10_000
                }
            }
        )


Okay, let's analyze the image step by step:

1.  **Subject:** The central figure is a young person, likely a boy or teenager.
2.  **Activity:** The person is sitting on the floor, leaning on a small table and writing on a piece of paper. They seem focused on their task.
3.  **Setting:** The setting appears to be a simple, perhaps makeshift, workspace. There's a small table, and a pile of clothes or bags behind them, suggesting a basic or temporary setup.
4.  **Overall impression:** The image conveys a sense of diligence or studying, despite the less-than-ideal workspace conditions.

**Therefore, the image describes a person (likely a student) studying or writing in a simple, perhaps resource-constrained, environment.**

Response at: 21.5 tk/s


In [20]:
usr_prompt = graph_gen._load_prompt_fromfile(WORK_DIR / "data/prompts/graph_gen/usr_prompt.txt")
reply = graph_gen._load_prompt_fromfile(WORK_DIR / "data/prompts/graph_gen/format_instructions.txt")
# Run frame generation
graph_gen.streaming_frame_generation(
    client,
    VIDEO_DIR, 
    "outputs/out_file.jsonl", 
    video_info=video_ids,
    usr_prompt=usr_prompt,
    reply=reply, 
    max_samples=10
)


Responses will be saved to: outputs/out_file.jsonl
Errors will be logged to: outputs/errors_20250523_17:32:30.txt
========================= Starting Response Generation =========================
Extracted frame at 0.10s: /tmp/tmpih8z81qz/frame_0000.jpg
Extracted frame at 3.58s: /tmp/tmpih8z81qz/frame_0001.jpg
Extracted frame at 7.16s: /tmp/tmpih8z81qz/frame_0002.jpg
Extracted frame at 10.74s: /tmp/tmpih8z81qz/frame_0003.jpg
Extracted frame at 14.32s: /tmp/tmpih8z81qz/frame_0004.jpg
Extracted frame at 17.91s: /tmp/tmpih8z81qz/frame_0005.jpg
Extracted frame at 21.49s: /tmp/tmpih8z81qz/frame_0006.jpg
Extracted frame at 25.07s: /tmp/tmpih8z81qz/frame_0007.jpg
Extracted frame at 28.65s: /tmp/tmpih8z81qz/frame_0008.jpg
Extracted frame at 32.13s: /tmp/tmpih8z81qz/frame_0009.jpg

Video: 0A8CF
 - interval: None-None
 - 10 frames.

Generating response for iteration 1 - id: 0A8CF
Okay, let's break down the image and describe the objects and their relationships.

**Objects Detected:**

1.  **Young

In [18]:
import prompt_formatters

prompt_formatter = prompt_formatters.OpenEndedPrompt("Q: {question}\nSTSG: {stsg}")
dataset = ollama_manager.PromptDataset("outputs/qa.json", prompt_formatter, "outputs/out_file.jsonl")


OSError: No such file or directory: 'outputs/qa.json'

In [19]:
dataset[0]


{'qid': 0,
 'prompt': 'Q: What is the boy holding?\nSTSG: \nFrame 0:\n\nyoung_man ---- sitting_at ---- table\ntable ---- supporting ---- notebook\nnotebook ---- containing ---- pen\npen ---- resting_on ---- notebook\nnotebook ---- positioned_in_front_of ---- young_man\npen ---- held_by ---- young_man\ntable ---- positioned_behind ---- young_man\nbag ---- hanging_behind ---- table\nbag ---- positioned_to_right_of ---- table\nbag ---- positioned_slightly_behind ---- young_man\ntable ---- on_floor ---- floor\nyoung_man ---- looking_at ---- notebook\nbag ---- hanging_on_wall ---- wall\n\nFrame 1:\n\nyoung_man ---- sitting_at ---- wooden_table\nwooden_table ---- on ---- floor\nyoung_man ---- wearing ---- white_tshirt\nyoung_man ---- wearing ---- black_shorts\nwhite_tshirt ---- part_of ---- young_man\nblack_shorts ---- part_of ---- young_man\nwhite_tshirt ---- adjacent_to ---- wooden_table\nblack_shorts ---- adjacent_to ---- wooden_table\nwooden_table ---- supporting ---- books\nbooks ---- o

In [11]:
cmd = f"{WORK_DIR}/src/graph_gen.py \
    --model gemma3:4b \
    --video-dir {WORK_DIR}/data/datasets/action-genome/Charades_v1_480 \
    --videos-metadata {WORK_DIR}/data/datasets/STAR/STAR_annotations/STAR_val.json\
    --output-file {WORK_DIR}/notebooks/outputs/out_file.jsonl \
    --usr-prompt {WORK_DIR}/data/prompts/graph_gen/usr_prompt.txt \
    --auto-reply {WORK_DIR}/data/prompts/graph_gen/format_instructions.txt \
    --max-samples 1"
    
%run {cmd}


=== Loading file with videos metadata: /home/lusha/star_code/data/datasets/STAR/STAR_annotations/STAR_val.json
Responses will be saved to: /home/lusha/star_code/notebooks/outputs/out_file.jsonl
Errors will be logged to: outputs/errors_20250520_14:34:46.txt
========================= Starting Response Generation =========================
Extracted frame at 15.35s: /tmp/tmpoj94_j3t/frame_0000.jpg

Video: 6H78U
 - interval: 11.1-19.6
 - 1 frames.

Generating response for iteration 1 - id: 6H78U
Okay, let's break down the image and analyze the objects and their relationships.

**Objects Detected:**

1.  **Man:** A young man is the primary subject, standing in the image.
2.  **Shirt:** He is wearing a white t-shirt.
3.  **Shorts:** He is wearing dark blue athletic shorts.
4.  **Sofa:** A large, ornate, brown and beige patterned sofa dominates the background. It has a high back and rolled arms.
5.  **Cushions:** Several cushions are arranged on the sofa, including a large, patterned cushion a

KeyboardInterrupt: 

In [16]:
import json

save_file = WORK_DIR / "notebooks/outputs/out_file.jsonl"

data = []
with open(save_file, 'r') as f:
    data = [json.loads(line) for line in f.readlines()]


In [13]:
data[0].keys()


dict_keys(['video_id', 'start', 'end', 'chat_history', 'stsg'])

In [14]:
data[0]['video_id'], data[0]['start'], data[0]['end']


('6H78U', 15.6, 22.7)

In [15]:
data[1]['video_id'], data[1]['start'], data[1]['end']


('RNLTR', 11.6, 16.4)

In [12]:
stsg_star = []
with open(f"{WORK_DIR}/data/datasets/STAR_QA_and_stsg_val.json") as f:
    stsg_star = json.load(f)


star_data = []
with open(f"{WORK_DIR}/data/datasets/STAR/STAR_annotations/STAR_val.json", 'r') as f:
    star_data = json.load(f)


In [13]:
same_order = True
for i, j in zip(stsg_star, star_data):
    test = i['question_id'] == j['question_id']
    same_order = same_order and test

same_order


True

In [10]:
def get_vocab_map(label_dir):
    vocab_map = dict()

    with open(label_dir) as in_file:
        for line in in_file.readlines():
            mapping = line.strip('\n')
            key, val = mapping.split(' ')
            vocab_map[key] = val
    
    return vocab_map

obj_vocab = get_vocab_map(WORK_DIR / 'data/datasets/STAR/STAR_annotations/class_maps/object_classes.txt')
rel_vocab = get_vocab_map(WORK_DIR / 'data/datasets/STAR/STAR_annotations/class_maps/relationship_classes.txt')

def generate_stsg(q_data):
    frame_ids = sorted(q_data['situations'].keys())

    stsg = []
    for f in frame_ids:
        frame_sg = []
        rels = q_data['situations'][f]['rel_labels']
        
        frame_sg.append(f"Frame {f}\n")
        for rel_pair, rel in zip(q_data['situations'][f]['rel_pairs'], rels):
            obj1, obj2 = rel_pair
            frame_sg.append(f'\t{obj_vocab[obj1]} --- {rel_vocab[rel]} --- {obj_vocab[obj2]}\n')
        
        frame_sg = ''.join(frame_sg)
        stsg.append(frame_sg)
    
    return stsg


In [15]:
print(''.join(generate_stsg(star_data[0])))


Frame 000198
	person --- on_the_side_of --- clothes
Frame 000202
	person --- in_front_of --- clothes
Frame 000205
	person --- in_front_of --- clothes
	person --- in_front_of --- blanket
Frame 000206
	person --- in_front_of --- towel
	person --- in_front_of --- clothes
	person --- in_front_of --- blanket
	person --- on_the_side_of --- blanket
Frame 000212
	person --- on_the_side_of --- towel
	person --- in_front_of --- towel
	person --- in_front_of --- clothes
	person --- on_the_side_of --- blanket
Frame 000217
	person --- in_front_of --- towel
	person --- on_the_side_of --- clothes
	person --- in_front_of --- blanket
	person --- on_the_side_of --- blanket
Frame 000218
	person --- in_front_of --- clothes
	person --- in_front_of --- blanket
	person --- on_the_side_of --- blanket
Frame 000221
	person --- in_front_of --- towel
	person --- in_front_of --- clothes
	person --- in_front_of --- blanket
Frame 000223
	person --- in_front_of --- towel
	person --- in_front_of --- clothes
	person --

In [19]:
new_stsg = []
for el in star_data:
    entry = {
        'question_id': el['question_id'],
        'video_id': el['video_id'],
        'start': el['start'],
        'end': el['end'],
        'stsg': ''.join(generate_stsg(el)),

    }
    new_stsg.append(entry)


In [20]:
print(new_stsg[0])


{'question_id': 'Interaction_T1_13', 'video_id': '6H78U', 'start': 11.1, 'end': 19.6, 'stsg': 'Frame 000198\n\tperson --- on_the_side_of --- clothes\nFrame 000202\n\tperson --- in_front_of --- clothes\nFrame 000205\n\tperson --- in_front_of --- clothes\n\tperson --- in_front_of --- blanket\nFrame 000206\n\tperson --- in_front_of --- towel\n\tperson --- in_front_of --- clothes\n\tperson --- in_front_of --- blanket\n\tperson --- on_the_side_of --- blanket\nFrame 000212\n\tperson --- on_the_side_of --- towel\n\tperson --- in_front_of --- towel\n\tperson --- in_front_of --- clothes\n\tperson --- on_the_side_of --- blanket\nFrame 000217\n\tperson --- in_front_of --- towel\n\tperson --- on_the_side_of --- clothes\n\tperson --- in_front_of --- blanket\n\tperson --- on_the_side_of --- blanket\nFrame 000218\n\tperson --- in_front_of --- clothes\n\tperson --- in_front_of --- blanket\n\tperson --- on_the_side_of --- blanket\nFrame 000221\n\tperson --- in_front_of --- towel\n\tperson --- in_front_

In [21]:
with  open(f"{WORK_DIR}/data/datasets/STAR_verbalized_stsg_val.json", 'w') as f:
    json.dump(new_stsg, f, indent=4)
